# Assignment 1: Prompt-Only Pricing Agent (Baseline)

## Objective
Build a simple LLM price recommender using **prompts only** - no tools, no data lookup, just pure LLM reasoning!

## Requirements
**User Input:**
- Cost Price
- Current Price
- Target Margin (%)
- Competitor Price
- Price Elasticity (Low/Medium/High)

**Agent Action:**
- Computes recommended price using verbal reasoning
- No tools, no data lookup
- Clear explanation of reasoning

## ️Setup & Dependencies

First, let's install the required packages and set up our environment.

In [1]:
# Install required packages
!pip install -q langchain langchain-groq

In [2]:
# Import required libraries
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage
import os
import getpass

In [3]:
# Set up your Groq API key
print("🔑 Please enter your Groq API key:")
print("(You can get one free at: https://console.groq.com/)")
groq_api_key = getpass.getpass("Groq API Key: ")
os.environ["GROQ_API_KEY"] = groq_api_key
print("✅ API key set successfully!")

🔑 Please enter your Groq API key:
(You can get one free at: https://console.groq.com/)
✅ API key set successfully!


## Initialize the Language Model

Let's set up our LLM for pricing recommendations.

In [4]:
# TODO: Initialize ChatGroq with appropriate parameters
# Hint: llm = ChatGroq() - use model, temperature, max_tokens etc

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
    max_tokens=1000,
    api_key=groq_api_key
)

## Prompt Engineering for Pricing

Now let's create effective prompts for our pricing agent. We'll start with a basic approach and then improve it.

### Basic Prompt

**Task:** Create a basic pricing prompt template. Fill in the template below.

In [5]:
# TODO: Complete this basic pricing prompt template
# HINT: basic_pricing_prompt = PromptTemplate() # Use input_varialbles, and a template string
# Make sure template can handle cost_price, current_price, target_margin, competitor_price, price_elasticity as variables
basic_pricing_prompt = PromptTemplate(
    input_variables=["cost_price", "current_price", "target_margin", "competitor_price", "price_elasticity"],
    template= "You are a retail pricing agent. Recommend an optimal selling price balancing price-elasticity={price_elasticity}, margin:{target_margin}, competitor-price:{competitor_price}, cost={cost_price} and current-price={current_price}. Show reasoning and keep response short."
)

### Test the Basic Prompt

Let's test our basic prompt with the example from the assignment.

In [6]:
# Test with the assignment example
test_input = {
    "cost_price": 400,
    "current_price": 599,
    "target_margin": 25,
    "competitor_price": 579,
    "price_elasticity": "Medium"
}

# TODO: Create chain and get response
basic_chain = basic_pricing_prompt | llm
basic_response = basic_chain.invoke(test_input)

print("📊 Basic Pricing Recommendation:")
print(basic_response.content)

📊 Basic Pricing Recommendation:
To determine the optimal selling price, we'll consider the price-elasticity, margin, competitor price, and cost.

Given:
- Price-elasticity: Medium (neither highly elastic nor inelastic)
- Margin: 25% (desired margin)
- Competitor price: $579
- Cost: $400
- Current price: $599

Since the price-elasticity is medium, we can adjust the price without significantly affecting demand. 

To achieve a 25% margin, the optimal selling price should be:
Optimal price = (Cost + Desired margin) / (1 - Desired margin)
Optimal price = ($400 + ($400 * 0.25)) / (1 - 0.25)
Optimal price = $500

However, considering the competitor price ($579) and the current price ($599), we should adjust the optimal price to be competitive while maintaining the desired margin.

Recommended optimal selling price: $579 (competitor price) is too low for a 25% margin, so we'll aim for a price between $579 and $599.
Recommended optimal selling price: $587 (a balance between maintaining the desi

### Improved Prompt with Chain-of-Thought

**Task:** Create an improved prompt that uses chain-of-thought reasoning for better explanations.

In [7]:
# TODO: Create an improved prompt with step-by-step reasoning
# Hint: improved_pricing_prompt = PromptTemplate() # input_variables and template.
# Use COT style prompting -
# ANALYSIS STEPS:
# 1. Calculate minimum price based on target margin
# 2. Analyze competitive positioning
# 3. Consider price elasticity impact
# 4. Recommend optimal price
improved_pricing_prompt = PromptTemplate(
    input_variables=["cost_price", "current_price", "target_margin", "competitor_price", "price_elasticity"],
    template="""
   You are a retail pricing agent.
   Recommend an optimal selling price following the factors and steps below:

   price elasticity={price_elasticity}
   margin:{target_margin}
   competitor-price:{competitor_price}
   cost={cost_price}
   current-price={current_price}

   Follow these analysis steps:
    1. Calculate minimum price based on target margin
    2. Analyze competitive positioning
    3. Consider price elasticity impact
    4. Recommend optimal price
   """)

In [8]:
# TODO: Test the improved prompt with the same data
improved_chain = improved_pricing_prompt | llm
improved_response = improved_chain.invoke(test_input)

print("📈 Improved Pricing Analysis:")
print(improved_response.content)

📈 Improved Pricing Analysis:
To determine the optimal selling price, I'll follow the steps you provided.

**Step 1: Calculate minimum price based on target margin**

To calculate the minimum price, we need to divide the cost by (1 - target margin).

Target margin = 25% = 0.25
Cost = $400

Minimum price = Cost / (1 - Target margin)
= $400 / (1 - 0.25)
= $400 / 0.75
= $533.33

**Step 2: Analyze competitive positioning**

The competitor's price is $579. Since our current price is $599, we are positioned above the competitor. To stay competitive, we may want to consider lowering our price.

**Step 3: Consider price elasticity impact**

Price elasticity is medium, which means that a small price change will have a moderate impact on demand. To balance the need to stay competitive with the potential impact on demand, we may want to consider a price reduction that is not too drastic.

**Step 4: Recommend optimal price**

Considering the minimum price ($533.33), competitive positioning, and pri

## Class-Based Pricing Agent

Now let's create a reusable class-based pricing agent, similar to the PhysicsTeacherAgent structure.

In [9]:
class PricingAgent:
    def __init__(self):
        # TODO: Initialize the LLM
        self.llm = ChatGroq(
          model="llama-3.1-8b-instant",
          temperature=0.7,
          max_tokens=1000,
          api_key=groq_api_key
        )

        # TODO: Create the system message for pricing agent persona
        self.system_message = SystemMessage(content="""
        You are a retail pricing agent who provides reasoning in a short and easy to follow manner.
        For arriving at the optimal price, you follow these analysis steps:
          1. Calculate minimum price based on target margin
          2. Analyze competitive positioning
          3. Consider price elasticity impact
          4. Recommend optimal price
        """)

        # TODO: Create the pricing prompt template
        self.pricing_template =  PromptTemplate(
          input_variables=["cost_price", "current_price", "target_margin", "competitor_price", "price_elasticity"],
          template="""
            Recommend an optimal selling price following the factors:

            price elasticity= {price_elasticity}
            margin = {target_margin}
            competitor price = {competitor_price}
            cost = {cost_price}
            current price = {current_price}
          """)
        pass

    def get_price_recommendation(self, cost_price, current_price, target_margin, competitor_price, price_elasticity):
        """Get price recommendation with detailed analysis"""
        # TODO: Implement the price recommendation logic
        messages = [
            self.system_message,
            HumanMessage(content=self.pricing_template.format(
                cost_price=cost_price,
                current_price=current_price,
                target_margin=target_margin,
                competitor_price=competitor_price,
                price_elasticity=price_elasticity
            ))
        ]

        response = self.llm.invoke(messages)
        return response.content

    def quick_price_check(self, cost_price, target_margin, competitor_price):
        """Quick price check with minimal inputs"""
        quick_prompt = f"""Quick pricing check:
        Cost: ${cost_price}, Target Margin: {target_margin}%, Competitor: ${competitor_price}

        Provide a quick price recommendation with brief reasoning."""

        response = self.llm.invoke([HumanMessage(content=quick_prompt)])
        return response.content

print("🏗️ Initializing Pricing Agent...")
pricing_agent = PricingAgent()
print("✅ Pricing Agent Ready!")

🏗️ Initializing Pricing Agent...
✅ Pricing Agent Ready!


## Testing Our Pricing Agent

Let's test our pricing agent with various scenarios.

In [10]:
# Test with the assignment example
print("Testing Assignment Example:")
print("=" * 50)

result = pricing_agent.get_price_recommendation(
    cost_price=400,
    current_price=599,
    target_margin=25,
    competitor_price=579,
    price_elasticity="Medium"
)

print(result)

Testing Assignment Example:
To determine the optimal selling price, I'll follow the analysis steps.

**Step 1: Calculate minimum price based on target margin**

Target margin = 25%
Cost = 400
Minimum price = (1 + 0.25) * Cost
Minimum price = 1.25 * 400
Minimum price = 500

**Step 2: Analyze competitive positioning**

Competitor price = 579

Since the competitor price is above the current price of 599, we may need to adjust our price to be more competitive. However, since the competitor price is also above the minimum price calculated, we still have some room to increase the price.

**Step 3: Consider price elasticity impact**

Price elasticity = Medium
Medium price elasticity means that the demand for the product is moderately sensitive to price changes.

To maintain demand, we can't increase the price too much. Considering the competitor price and the minimum price, I'll keep the price adjustment moderate.

**Step 4: Recommend optimal price**

Based on the analysis:

- Current price =

In [11]:
# TODO: Test with different scenarios - Add your own test cases!

# Test Case 1: High elasticity scenario
print("\nTest Case 1: High Elasticity Scenario")
print("=" * 50)
result1 = pricing_agent.get_price_recommendation(
    cost_price=100,
    current_price=200,
    target_margin=30,
    competitor_price=180,
    price_elasticity="High"
)
print(result1)

# Test Case 2: Low elasticity scenario
print("\nTest Case 2: Low Elasticity Scenario")
print("=" * 50)
result2 = pricing_agent.get_price_recommendation(
    cost_price=50,
    current_price=100,
    target_margin=40,
    competitor_price=120,
    price_elasticity="Low"
)
print(result2)


Test Case 1: High Elasticity Scenario
To find the optimal selling price, I'll follow the steps:

**Step 1: Calculate minimum price based on target margin**

Target margin = 30% 
Cost = 100
Minimum selling price = Cost / (1 - Target margin) 
Minimum selling price = 100 / (1 - 0.3) 
Minimum selling price = 100 / 0.7 
Minimum selling price = 142.86

**Step 2: Analyze competitive positioning**

Competitor price = 180
Since the competitor price is higher than the minimum selling price, we can consider this as a reference point.

**Step 3: Consider price elasticity impact**

Price elasticity = High
High price elasticity means small changes in price will result in large changes in demand.
To maximize revenue, we should aim for a price that's close to the current price (200) but still competitive.

**Step 4: Recommend optimal price**

Considering the competitor price and high price elasticity, we can adjust our price downward to stay competitive.
Optimal price = Current price * (Competitor pr

In [12]:
# Test the quick price check function
print("⚡ Quick Price Check Test:")
print("=" * 30)

quick_result = pricing_agent.quick_price_check(
    cost_price=250,
    target_margin=20,
    competitor_price=350
)
print(quick_result)

⚡ Quick Price Check Test:
To determine the price, let's calculate the desired selling price based on the target margin.

Cost: $250
Target Margin: 20% (or 0.20)
Target Selling Price = (Cost / (1 - Target Margin)) = ($250 / (1 - 0.20)) = $250 / 0.80 = $312.50

This price is higher than the competitor's price ($350). To stay competitive, we could consider lowering the target selling price.


## Experiment with Different Prompting Strategies

Try different prompting approaches and compare the results.

In [13]:
few_shot_template = PromptTemplate(
    input_variables=["cost_price", "current_price", "target_margin", "competitor_price", "price_elasticity"],
    template="""You are a pricing expert. Here are some examples of good pricing decisions:

Example 1:
Cost: $200, Current: $400, Target Margin: 30%, Competitor: $380, Elasticity: Medium
Recommendation: $375 (maintains margin above 30%, competitive with market, good for medium elasticity)

Example 2:
Cost: $100, Current: $180, Target Margin: 25%, Competitor: $200, Elasticity: Low
Recommendation: $190 (exceeds margin target, leverages low elasticity for higher profit)

Now analyze this case:
Cost: ${cost_price}, Current: ${current_price}, Target Margin: {target_margin}%, Competitor: ${competitor_price}, Elasticity: {price_elasticity}

Recommendation:"""
)

# Test few-shot approach
few_shot_chain = few_shot_template | llm
few_shot_result = few_shot_chain.invoke(test_input)

print("🎯 Few-Shot Prompting Result:")
print(few_shot_result.content)

🎯 Few-Shot Prompting Result:
To make a pricing recommendation, let's analyze the given data:

- Cost: $400
- Current price: $599
- Target Margin: 25%
- Competitor price: $579
- Elasticity: Medium

First, let's calculate the current margin:
Current margin = (Current price - Cost) / Cost
Current margin = ($599 - $400) / $400
Current margin = $199 / $400
Current margin = 0.4975 or 49.75%

Since the target margin is 25%, the current price is above the target margin. To adjust the price and meet the target margin, we need to calculate the new price.

New price = (Target Margin / Current margin) * Current price
New price = (0.25 / 0.4975) * $599
New price ≈ 1.503 * $599
New price ≈ $902.97

However, this new price is not competitive with the market, and it's also higher than the competitor's price. Considering the medium elasticity, we should aim for a price that balances the target margin with market competitiveness.

Let's calculate the price that maintains the target margin while being co

## Compare Different Approaches

Let's compare the different prompting strategies we've implemented.

In [14]:
# TODO: Create a comparison of all approaches
def compare_pricing_approaches(cost_price, current_price, target_margin, competitor_price, price_elasticity):
    """Compare different prompting approaches for the same input"""

    print(f"📊 PRICING COMPARISON FOR:")
    print(f"Cost: ${cost_price}, Current: ${current_price}, Target Margin: {target_margin}%")
    print(f"Competitor: ${competitor_price}, Elasticity: {price_elasticity}")
    print("="*60)

    # Basic approach
    basic_result = basic_chain.invoke({
        "cost_price": cost_price,
        "current_price": current_price,
        "target_margin": target_margin,
        "competitor_price": competitor_price,
        "price_elasticity": price_elasticity
    })

    # Improved approach
    improved_result = improved_chain.invoke({
        "cost_price": cost_price,
        "current_price": current_price,
        "target_margin": target_margin,
        "competitor_price": competitor_price,
        "price_elasticity": price_elasticity
    })

    # Class-based approach
    class_result = pricing_agent.get_price_recommendation(
        cost_price, current_price, target_margin, competitor_price, price_elasticity
    )

    print("🔴 BASIC APPROACH:")
    print(basic_result.content + "...\n")

    print("🟡 IMPROVED APPROACH:")
    print(improved_result.content + "...\n")

    print("🟢 CLASS-BASED APPROACH:")
    print(class_result + "...")

# Run comparison
compare_pricing_approaches(400, 599, 25, 579, "Medium")

📊 PRICING COMPARISON FOR:
Cost: $400, Current: $599, Target Margin: 25%
Competitor: $579, Elasticity: Medium
🔴 BASIC APPROACH:
To determine the optimal selling price, we'll consider the price-elasticity, margin, competitor price, and cost.

Given:
- Price-elasticity: Medium (neither highly elastic nor inelastic)
- Margin: 25% (desired margin)
- Competitor price: $579
- Cost: $400
- Current price: $599

Since the price-elasticity is medium, we can adjust the price without significantly affecting demand. 

To achieve a 25% margin, the optimal selling price should be:
Optimal price = (Cost + Desired margin) / (1 - Desired margin)
Optimal price = ($400 + ($400 * 0.25)) / (1 - 0.25)
Optimal price = $500

However, considering the competitor price ($579) and the current price ($599), we should adjust the optimal price to be competitive while maintaining the desired margin.

Recommended optimal selling price: $579 (competitor price) is too low for a 25% margin, so we'll aim for a price between